In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import seaborn as sns
import numpy as np

%matplotlib inline

In [2]:
hm_days = 7


def process_data_for_labels(ticker):
    df = pd.read_csv('sp500_joined_adjcloses.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)

    for i in range(1, hm_days+1):
        df['{}_{}d'.format(ticker, i)] = (
            df[ticker].shift(-i) - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    return tickers, df

In [3]:
# process_data_for_labels('XOM')

In [4]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > requirement:
            return 1
        elif col < -requirement:
            return -1
    return 0

In [5]:
def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)

    df['{}_target'.format(ticker)] = list(map(buy_sell_hold, *[df['{}_{}d'.format(ticker, i)]
                                                               for i in range(1, hm_days+1)]))

#                                               df['{}_1d'.format(ticker)],
#                                               df['{}_2d'.format(ticker)],
#                                               df['{}_3d'.format(ticker)],
#                                               df['{}_4d'.format(ticker)],
#                                               df['{}_5d'.format(ticker)],
#                                               df['{}_6d'.format(ticker)],
#                                               df['{}_7d'.format(ticker)]))

    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:', Counter(str_vals))

    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)

    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)

    X, y = df_vals.values, df['{}_target'.format(ticker)].values
    return X, y, df

In [6]:
X, y, df = extract_featuresets('XOM')

Data spread: Counter({'1': 1715, '-1': 1456, '0': 1108})
